## Linear Regression

**(Multi-Linear Regression)**

In [1]:
import pandas as pd
import numpy as np


In [2]:
data = pd.read_csv("C:\\Users\\user\\Downloads\\Amazon Sale Report.csv\\Amazon Sale Report.csv")

C:\Users\user\AppData\Local\Temp\ipykernel_10952\369179484.py:1: DtypeWarning: Columns (23) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("C:\\Users\\user\\Downloads\\Amazon Sale Report.csv\\Amazon Sale Report.csv")


In [3]:
data

,index,Order ID,Date,Status,Fulfilment,Sales Channel,ship-service-level,Style,SKU,Category,...,currency,Amount,ship-city,ship-state,ship-postal-code,ship-country,promotion-ids,B2B,fulfilled-by,Unnamed: 22
0,0,405-8078784-5731545,04-30-22,Cancelled,Merchant,Amazon.in,Standard,SET389,SET389-KR-NP-S,Set,...,INR,647.62,MUMBAI,MAHARASHTRA,400081.0,IN,NaN,False,Easy Ship,NaN
1,1,171-9198151-1101146,04-30-22,Shipped - Delivered to Buyer,Merchant,Amazon.in,Standard,JNE3781,JNE3781-KR-XXXL,kurta,...,INR,406.00,BENGALURU,KARNATAKA,560085.0,IN,Amazon PLCC Free-Financing Universal Merchant ...,False,Easy Ship,NaN
2,2,404-0687676-7273146,04-30-22,Shipped,Amazon,Amazon.in,Expedited,JNE3371,JNE3371-KR-XL,kurta,...,INR,329.00,NAVI MUMBAI,MAHARASHTRA,410210.0,IN,IN Core Free Shipping 2015/04/08 23-48-5-108,True,NaN,NaN
3,3,403-9615377-8133951,04-30-22,Cancelled,Merchant,Amazon.in,Standard,J0341,J0341-DR-L,Western Dress,...,INR,753.33,PUDUCHERRY,PUDUCHERRY,605008.0,IN,NaN,False,Easy Ship,NaN
4,4,407-1069790-7240320,04-30-22,Shipped,Amazon,Amazon.in,Expedited,JNE3671,JNE3671-TU-XXXL,Top,...,INR,574.00,CHENNAI,TAMIL NADU,600073.0,IN,NaN,False,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128970,128970,406-6001380-7673107,05-31-22,Shipped,Amazon,Amazon.in,Expedited,JNE3697,JNE3697-KR-XL,kurta,...,INR,517.00,HYDERABAD,TELANGANA,500013.0,IN,NaN,False,NaN,False
128971,128971,402-9551604-7544318,05-31-22,Shipped,Amazon,Amazon.in,Expedited,SET401,SET401-KR-NP-M,Set,...,INR,999.00,GURUGRAM,HARYANA,122004.0,IN,IN Core Free Shipping 2015/04/08 23-48-5-108,False,NaN,False
128972,128972,407-9547469-3152358,05-31-22,Shipped,Amazon,Amazon.in,Expedited,J0157,J0157-DR-XXL,Western Dress,...,INR,690.00,HYDERABAD,TELANGANA,500049.0,IN,NaN,False,NaN,False
128973,128973,402-6184140-0545956,05-31-22,Shipped,Amazon,Amazon.in,Expedited,J0012,J0012-SKD-XS,Set,...,INR,1199.00,Halol,Gujarat,389350.0,IN,IN Core Free Shipping 2015/04/08 23-48-5-108,False,NaN,False


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 128975 entries, 0 to 128974
Data columns (total 24 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   index               128975 non-null  int64  
 1   Order ID            128975 non-null  object 
 2   Date                128975 non-null  object 
 3   Status              128975 non-null  object 
 4   Fulfilment          128975 non-null  object 
 5   Sales Channel       128975 non-null  object 
 6   ship-service-level  128975 non-null  object 
 7   Style               128975 non-null  object 
 8   SKU                 128975 non-null  object 
 9   Category            128975 non-null  object 
 10  Size                128975 non-null  object 
 11  ASIN                128975 non-null  object 
 12  Courier Status      122103 non-null  object 
 13  Qty                 128975 non-null  int64  
 14  currency            121180 non-null  object 
 15  Amount              121180 non-nul

## Data Transformation

In [5]:
# first drop the most unwanted columns
DROP_COLS = [
    "index",
    "Order ID",           # pure identifier
    "SKU",                # high cardinality ID
    "promotion-ids",      # sparse + leakage prone
    "Unnamed: 22",
    "ASIN",
    "fulfilled-by",
    "ship-postal-code",
    "currency",
    "Size"
]

data = data.drop(columns=[c for c in DROP_COLS if c in data.columns])


In [6]:
data.columns

Index(['Date', 'Status', 'Fulfilment', 'Sales Channel ', 'ship-service-level',
       'Style', 'Category', 'Courier Status', 'Qty', 'Amount', 'ship-city',
       'ship-state', 'ship-country', 'B2B'],
      dtype='object')

In [7]:
# rename some columns
data.rename(columns={'Sales Channel ':'Sales Channel'}, inplace=True)

In [8]:
data.columns

Index(['Date', 'Status', 'Fulfilment', 'Sales Channel', 'ship-service-level',
       'Style', 'Category', 'Courier Status', 'Qty', 'Amount', 'ship-city',
       'ship-state', 'ship-country', 'B2B'],
      dtype='object')

In [9]:
# extract some important columsn from existing columns
data["Date"] = pd.to_datetime(data["Date"], dayfirst=True, errors="coerce")

data["order_day"] = data["Date"].dt.day_name()
data["order_month"] = data["Date"].dt.month_name()
data["order_weekday"] = data["Date"].dt.weekday
data["is_weekend"] = data["order_weekday"].isin([5,6]).astype(int)


# again drop the irrelevant columns
data = data.drop(columns=["Date"])


C:\Users\user\AppData\Local\Temp\ipykernel_4540\2998258809.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data["Date"] = pd.to_datetime(data["Date"], dayfirst=True, errors="coerce")


In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 128975 entries, 0 to 128974
Data columns (total 17 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   Status              128975 non-null  object 
 1   Fulfilment          128975 non-null  object 
 2   Sales Channel       128975 non-null  object 
 3   ship-service-level  128975 non-null  object 
 4   Style               128975 non-null  object 
 5   Category            128975 non-null  object 
 6   Courier Status      122103 non-null  object 
 7   Qty                 128975 non-null  int64  
 8   Amount              121180 non-null  float64
 9   ship-city           128942 non-null  object 
 10  ship-state          128942 non-null  object 
 11  ship-country        128942 non-null  object 
 12  B2B                 128975 non-null  bool   
 13  order_day           128975 non-null  object 
 14  order_month         128975 non-null  object 
 15  order_weekday       128975 non-nul

till now, feature extraction is done!

irrelevant cols are removed from dataset!

Now, we will build a ML pipeline to handle null-values, scale the features, encode the features,etc.

but having different pipeline for numerical and categorical columns!/

## Feature Engineering

In [11]:
# seperating numerical and categorical columns
numerical_cols = data.select_dtypes(include="number").columns
print("numerical_cols:\n", numerical_cols)
print()

categorical_cols = data.select_dtypes(exclude="number").columns
print("catgeorical_cols:\n", categorical_cols)

numerical_cols:
 Index(['Qty', 'Amount', 'order_weekday', 'is_weekend'], dtype='object')

catgeorical_cols:
 Index(['Status', 'Fulfilment', 'Sales Channel', 'ship-service-level', 'Style',
       'Category', 'Courier Status', 'ship-city', 'ship-state', 'ship-country',
       'B2B', 'order_day', 'order_month'],
      dtype='object')


In [12]:
numerical_cols = numerical_cols.drop(['Amount'])
numerical_cols

Index(['Qty', 'order_weekday', 'is_weekend'], dtype='object')

**a) This is the stepwise execution of all phases, to know what's happening; finally we will combine all things!**

In [13]:
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler


In [14]:
# numerical pipeline:
# fill with median values, then -> scale the values between -3 and +3
num_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

# categorical pipeline
# fill with mode value ,then -> convert to numbers using OHE
cat_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("encoder", OneHotEncoder(
        handle_unknown="ignore",
        min_frequency=0.01,   # reduces noise / high cardinality
    ))
])


In [15]:
cat_pipeline

Pipeline(steps=[('imputer', SimpleImputer(strategy='most_frequent')),
                ('encoder',
                 OneHotEncoder(handle_unknown='ignore', min_frequency=0.01))])

In [16]:
# wrapping all pipelines in a preprocessor object -> 
# where we have a numerical pipeline for numerical cols, and a categorical pipeline for categorical cols
preprocessor = ColumnTransformer([
    ("num", num_pipeline, numerical_cols),
    ("cat", cat_pipeline, categorical_cols)
])


In [17]:
preprocessor

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='median')),
                                                 ('scaler', StandardScaler())]),
                                 Index(['Qty', 'order_weekday', 'is_weekend'], dtype='object')),
                                ('cat',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('encoder',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                min_frequency=0.01))]),
                                 Index(['Status', 'Fulfilment', 'Sales Channel', 'ship-service-level', 'Style',
       'Category', 'Courier Status', 'ship-city', 'ship-state', 'ship-country',
       'B2B', 'order_day', 'order_month'],
      dtype='object'))])

**b) either choose (a) or this step, here we merging all steps in a function:**


In [19]:

# build a function -> general purpose -> to handle ml preprocessings
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer

def build_preprocessor(num_cols, cat_cols, scale_numeric=True):
    
    num_steps = [
        ("imputer", SimpleImputer(strategy="median"))
    ]
    
    if scale_numeric:
        num_steps.append(("scaler", StandardScaler()))
    
    num_pipeline = Pipeline(num_steps)

    cat_pipeline = Pipeline([
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("encoder", OneHotEncoder(handle_unknown="ignore"))
    ])

    return ColumnTransformer([
        ("num", num_pipeline, num_cols),
        ("cat", cat_pipeline, cat_cols)
    ])


In [20]:

# for linear models -> scaling is on!
preprocessor = build_preprocessor(
    num_cols=numerical_cols,
    cat_cols=categorical_cols,
    scale_numeric=True
)


**3. seperating all target and input variables**

In [21]:

TARGET = "Amount"   # regression target

X = data.drop(columns=[TARGET])
y = data[TARGET]


In [24]:
y.isna().sum()

np.int64(7795)

In [25]:
X.isna().sum()

Status                   0
Fulfilment               0
Sales Channel            0
ship-service-level       0
Style                    0
Category                 0
Courier Status        6872
Qty                      0
ship-city               33
ship-state              33
ship-country            33
B2B                      0
order_day                0
order_month              0
order_weekday            0
is_weekend               0
dtype: int64

In [26]:
y.fillna(value=data['Amount'].median(), inplace=True)


**4. split before preprocessing to avoid leakage.**

In [27]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42
)


**a) building model using pipeline**

In [28]:
from sklearn.linear_model import LinearRegression

model = Pipeline([
    ("preprocess", preprocessor),
    ("regressor", LinearRegression())
])

model.fit(X_train, y_train)


Pipeline(steps=[('preprocess',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  Index(['Qty', 'order_weekday', 'is_weekend'], dtype='object')),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('encoder',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  Index(['Status', 'Fulfilment', 'Sales Channel', 'ship-service-level', 'Style',
       'Category', 'Courier Status', 'ship-city', 'ship-state', 'ship-country',
       'B2B', 'order_day', 'order_month'],
      dtype='object'))])),
                ('regressor', LinearRegression())])

In [30]:
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

y_pred = model.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print("R2:", r2_score(y_test, y_pred))
print("MAE:", mean_absolute_error(y_test, y_pred))
print("RMSE:", rmse)

R2: 0.7917152475504555
MAE: 60.963304931211376
RMSE: 125.5825544066296


In [31]:
feature_names = preprocessor.get_feature_names_out()
# feature_names

**b) building model by first finding the processed data, then train it**

In [33]:
X_train_processed = preprocessor.fit_transform(X_train)
X_test_processed = preprocessor.transform(X_test)


In [34]:
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression()

lin_reg.fit(X_train_processed, y_train)


LinearRegression()

In [35]:
y_pred = lin_reg.predict(X_test_processed)

In [36]:
import numpy as np
from sklearn.metrics import mean_squared_error

rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print("R2:", r2_score(y_test, y_pred))
print("MAE:", mean_absolute_error(y_test, y_pred))
print("RMSE:", rmse)


R2: 0.7917152475504555
MAE: 60.963304931211376
RMSE: 125.5825544066296
